## Imports

https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/mp/jupyter/tutorials/Linear_Programming.ipynb


Integer Optimization:
- https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/mp/jupyter/tutorials/Beyond_Linear_Programming.ipynb

In [1]:
from docplex.mp.model import Model
import numpy as np
import pandas as pd

In [40]:
# load data

players = pd.read_csv("../data/processed-data/0_selection_players.csv", index_col=0)
n_players = players.shape[0]
salaries = pd.read_csv("../data/processed-data/0_selection_salary.csv", index_col=0,).drop('Player', axis=1).values
n_time = salaries.shape[1]
n_trials = int(salaries.shape[0]/n_players)
salaries = salaries.reshape((n_trials, n_players, n_time))
market_values = salaries/0.1

n_time = 1
salaries = salaries[:, :, :n_time]
market_values = market_values[:, :, :n_time]


# Set up all players
# use binary var cube
# (time, player, and simulation)


# set up goalkeeping separately
# set up DF, MF and Fw using separate dataset



In [41]:
salaries.shape

(3, 2822, 1)

In [42]:
# create model
ftcp = Model(name = "Rarita Football Team Composition")

max_age = 40
starting_budget = 15000000
rho = 0.05

time = np.arange(n_time)
trials = np.arange(n_trials)

In [43]:
# Decision variables
# assigned = ftcp.binary_var_matrix(keys1=players.index.values, keys2=time, keys3=trials, name = "assign_%s_%s_%s")
assigned = ftcp.binary_var_matrix(keys1=players.index.values, keys2=time, name = "assign_%s_%s")

budget = ftcp.continuous_var_list(keys = time, name = "Budget_%s", lb=0)
team_value = ftcp.continuous_var_list(keys = time, name = "Team Value_%s")

sunset_value = ftcp.continuous_var(name = "Sunset Value_")
optim_value = ftcp.continuous_var(name = "Expectation of Team Value")

In [44]:
# Non decision variables
# Total number of players
N_total = ftcp.integer_var_list(keys = time, name = 'Total number of Players_%s', lb=22, ub=25)

# Number of players in each role
N_gk = ftcp.integer_var_list(keys = time, name = "Number of goalkeepers_%s", lb=2, ub=3)
N_df = ftcp.integer_var_list(keys = time, name = "Number of defenders_%s", lb=8, ub=10)
N_mf = ftcp.integer_var_list(keys = time, name = "Number of midfielders_%s", lb=8, ub=10)
N_fw = ftcp.integer_var_list(keys = time, name = "Number of forwards_%s", lb=3, ub=4)       

In [45]:
# Set up Constraints for number of players
for year in time:
    
    previous_budget = starting_budget if year == 0 else budget[year-1]   
    ftcp.add_constraint(budget[year] == previous_budget 
                        - ftcp.sum(assigned[index, year] * salaries[0, index, year] for index in range(n_players)))
    
    # Team value = Player values - Wages + Borrowing - Lending
    ftcp.add_constraint(team_value[year] == ftcp.sum(assigned[index, year] * market_values[0, index, year] for index in range(n_players))/((1 + rho)**year)
                        - ftcp.sum(assigned[index, year] * salaries[0, index, year] for index in range(n_players)))
    
    ftcp.add_constraint(N_gk[year] == ftcp.sum(assigned[index, year] * player['GK'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_df[year] == ftcp.sum(assigned[index, year] * player['DF'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_mf[year] == ftcp.sum(assigned[index, year] * player['MF'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_fw[year] == ftcp.sum(assigned[index, year] * player['FW'] for index, player in players.iterrows()))
    ftcp.add_constraint(N_gk[year] + N_df[year] + N_mf[year] + N_fw[year] == N_total[year])

    for index, player in players.iterrows():
        ftcp.add_constraint(assigned[index, year] * (player['Age'] + year) <= max_age )
        
ftcp.add_constraint(sunset_value == ftcp.sum(assigned[index, time[-1]] * market_values[0, index, time[-1]] for index in range(n_players))/((1 + rho)**(n_time)))
ftcp.add_constraint(optim_value == ftcp.sum(team_value) + sunset_value)

docplex.mp.LinearConstraint[](Expectation of Team Value,EQ,Team Value_0+Sunset Value_)

In [51]:
ftcp.maximize(optim_value)
ftcp.print_information()

Model: Rarita Football Team Composition
 - number of variables: 2831
   - binary=2822, integer=5, continuous=4
 - number of constraints: 2831
   - linear=2831
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


In [52]:
ftcp.solve(log_output=True, clean_before_solve=True)
ftcp.solve_status

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 2824 rows and 2005 columns.
MIP Presolve added 132 rows and 132 columns.
MIP Presolve modified 28 coefficients.
Reduced MIP has 139 rows, 958 columns, and 3175 nonzeros.
Reduced MIP has 807 binaries, 150 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (4.80 ticks)
Found incumbent of value 2.7581952e+08 after 0.03 sec. (15.14 ticks)
Probing fixed 118 vars, tightened 165 bounds.
Probing time = 0.14 sec. (64.83 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 133 rows and 250 columns.
MIP Presolve added 152 rows and 152 columns.
Reduced MIP has 158 rows, 860 columns, and 2129 nonzeros.
Reduced MIP has 754 binaries, 105 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.08 sec. (1.99 ticks)
Probing time = 0.01 sec. (4.51 ticks)
Cover probing fixed 0 vars, tightened 34 bounds.
Tried aggregator 1 time.
Detecting symm

<JobSolveStatus.OPTIMAL_SOLUTION: 2>

In [56]:
ftcp.print_solution()

objective: 277857142.857
  "assign_2606_0"=1
  "assign_2609_0"=1
  "assign_2629_0"=1
  "assign_2630_0"=1
  "assign_2634_0"=1
  "assign_2635_0"=1
  "assign_2642_0"=1
  "assign_2648_0"=1
  "assign_2649_0"=1
  "assign_2650_0"=1
  "assign_2684_0"=1
  "assign_2686_0"=1
  "assign_2692_0"=1
  "assign_2704_0"=1
  "assign_2712_0"=1
  "assign_2753_0"=1
  "assign_2802_0"=1
  "assign_2805_0"=1
  "Team Value_0"=135000000.000
  "Sunset Value_"=142857142.857
  "Expectation of Team Value"=277857142.857
  "Total number of Players_0"=23
  "Number of goalkeepers_0"=3
  "Number of defenders_0"=8
  "Number of midfielders_0"=8
  "Number of forwards_0"=4


In [53]:
import docplex.mp.conflict_refiner as cr

ftcp.solve()
solve_status = ftcp.get_solve_status()
if solve_status.name in ['INFEASIBLE_SOLUTION', 'INFEASIBLE_OR_UNBOUNDED_SOLUTION']:
    cref = cr.ConflictRefiner()
    print('show some of the constraints that can be removed to arrive at a minimal conflict')
    cref_result = cref.refine_conflict(ftcp, display=True)  # display flag is to show the conflicts

In [54]:
cref_result.display()

conflict(s): 14
  - status: Member, LinearConstraint: Budget_0 == -26230000 assign_0_0 - 10180000 assign_1_0 - 270..
  - status: Member, LinearConstraint: Number of goalkeepers_0 == assign_13_0 + assign_14_0 + assign_15_0 + assign..
  - status: Member, LinearConstraint: Number of defenders_0 == assign_0_0 + assign_1_0 + assign_2_0 + assign_3_..
  - status: Member, LinearConstraint: Number of midfielders_0 == assign_17_0 + assign_18_0 + assign_19_0 + assign..
  - status: Member, LinearConstraint: Number of forwards_0 == assign_9_0 + assign_10_0 + assign_11_0 + assign_..
  - status: Member, LinearConstraint: Number of goalkeepers_0 + Number of defenders_0 .. == Total number of Players_0
  - status: Member, LinearConstraint: Budget_1 == -26269000 assign_0_1 - 10193000 assign_1_1 - 270..
  - status: Member, LinearConstraint: Number of goalkeepers_1 == assign_13_1 + assign_14_1 + assign_15_1 + assign..
  - status: Member, LinearConstraint: Number of defenders_1 == assign_0_1 + assign_1_1 + 

In [55]:
cref_result.as_output_table()

,Type,Status,Name,Expression
0,Constraint,Member,,Budget_0 == -26230000assign_0_0-10180000assign...
1,Constraint,Member,,Number of goalkeepers_0 == assign_13_0+assign_...
2,Constraint,Member,,Number of defenders_0 == assign_0_0+assign_1_0...
3,Constraint,Member,,Number of midfielders_0 == assign_17_0+assign_...
4,Constraint,Member,,Number of forwards_0 == assign_9_0+assign_10_0...
5,Constraint,Member,,Number of goalkeepers_0+Number of defenders_0+...
6,Constraint,Member,,Budget_1 == -26269000assign_0_1-10193000assign...
7,Constraint,Member,,Number of goalkeepers_1 == assign_13_1+assign_...
8,Constraint,Member,,Number of defenders_1 == assign_0_1+assign_1_1...
9,Constraint,Member,,Number of midfielders_1 == assign_17_1+assign_...
